In [571]:
import numpy as np
import numpy.matlib 
import math
import matplotlib.pyplot as plt
import scipy.stats

# Inspired by youtube channel D&D Optimised (now d4: D&D Deep Dive)

## Calculate average damage

### Source: D&D Optimised
https://www.youtube.com/watch?v=nk2SIaGXSYo

"The basic formula for calculating average damage is as follows:				
((Average Weapon Damage + Applicable per-hit Damage Modifiers)*(% chance to hit))+((Average Dice Damage on attack)*(% chance to crit))+any once per round damage													
													
Keep in mind that if you have multiple attacks, you need to run the above for each attack - you can simply multiply everything between the outermost parentheses by the # of attacks *if* those attacks are identical,													
but in the case of someone using a Polearm Master Feat, for example, they are not, as the regular attacks would use a D10 for the weapon, wheras the bonus action attack would use a D4 for the weapon."													

### Source: https://www.dndbeyond.com/sources/phb/combat#MakinganAttack
"To make an attack roll, roll a d20 and add the appropriate modifiers. If the total of the roll plus modifiers equals or exceeds the target's Armor Class (AC), the attack hits." 


## Detailed calculations

### Definitions
[roll]: Average Dice Roll
OAC: Opponents Armour Class

### Chance to hit and chance to miss

P(My hit was greater or equal than the Opponents AC) = P([roll]+ hit bonus >= OAC) => P([roll]>= OAC- hit bonus)

P(hit)=P([rol] >= OAC -hit bonus) = [ 20 -(OAC- hit bonus)+1]/20 = 1 -(OAC- hit bonus -1)/20 = 1 - P(miss)

P(miss) = (OAC- hit bonus -1)/20

### Chance to hit with advantage
P (hit on first roll but not on the second)+ P (hit on the second roll but not on the first)+ P(hit on both rolls)=  2  * [ 1 -(OAC- hit bonus -1)/20] * [ (OAC- hit bonus -1)/20]+ [ 1 -(OAC- hit bonus -1)/20] **2 (see also binomial distribution).

The shorter way is to calculate "1 - the chance of missing both rolls". 

P (miss both rolls) = [P(miss)] = [(OAC- hit bonus -1)/20]**2

P( hit with an advantage) = 1 - [(OAC- hit bonus -1)/20]**2

### Chance to hit with advantage and Elven Accuracy (or Lucky)
Elven accuracy, i.e roll one more time: 1- [(OAC- hit bonus -1)/20]**3

### Critical hit  (minimum chance to hit)
Crit hit (natural 20): p(critit)= 1/20

### Critical hit with advantage
Crit hit (natural 20): p(critit)= 1- (1- 1/20)**2

### Minimum chance to hit with advantage and Elven Accuracy (or Lucky)
Crit hit (natural 20): p(critit)= 1- (1- 1/20)**3

### Minimum chance to hit with Hexblade Curse on target (HCT)
Crit hit (19 or 20) p(critit HCT)= 1- (1-2/20)

In [572]:
1- (1-2/20)

0.09999999999999998

### Minimum chance to hit with advantage, Elven Accuracy (or Lucky) and Hexblade Curse on target (HCT)
Crit hit (19 or 20) p(critit HCT)= 1- (1-2/20)**3

In [573]:
1- (1-2/20)**3

0.2709999999999999

One way to obtain advantage as a Warlock is to get Devil's Sight and Cast Darkness on oneself (when multiclass with e.g. fighter, one may use the blind fighting style instead of Devil's Sight).

##  Calculating the Probablility of critical hits (rounded to 5 decimal digits).

In [574]:
import dnd_optimisation as dndopt # this is a python file containing the calcuations for critical hits

In [575]:
import importlib  #This is because I am making changes to the file all the time ;) 
importlib.reload(dndopt)  

<module 'dnd_optimisation' from '/Users/eleni/Python Scripts/DnD/dnd_optimisation.py'>

#### Setting advantage, luck_point (related to the lucky feat) and elven_accuracy to false gives the probabity of a natural critical hit (natural 20) 1/20

In [576]:
advantage=False; luck_point=False; elven_accuracy=False; hexblade_curse=False

In [577]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.05

#### Only advantage

In [578]:
advantage=True; luck_point=False; elven_accuracy=False; hexblade_curse=False

In [579]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.0975

#### Advantage and lucky

In [580]:
advantage=True; luck_point=True; elven_accuracy=False; hexblade_curse=False

In [581]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.14263

#### Hexblade curse with advantage

In [582]:
advantage=True; luck_point=True; elven_accuracy=False; hexblade_curse=True

In [583]:
dndopt.probability_of_crtical_hit(advantage,luck_point,elven_accuracy,hexblade_curse)

0.271

# Comparing different builds

## Warlock (Celestial Patron) with 2 levels of Fighter vs 2 levels of Paladin for maximal damage 

### Assuming the following Ability Scores

In [584]:
AbilityScores= numpy.array([19, 14, 12, 16, 8, 18])

In [585]:
Abilities=['STR', 'DEX', 'CON','INT','WIS', 'CHA']

### Calculate the Ability Score Modifiers

In [586]:
Modifiers = np.floor((AbilityScores - 10)/2)

In [587]:
Modifiers=Modifiers.astype(int)

### What is the Ability Score for Charisma?

In [588]:
AbilityScores[Abilities.index('CHA')]

18

### What is the Modifier for Charisma?

In [589]:
Modifiers[Abilities.index('CHA')]

4

### Define the Character's Level

In [590]:
Level=8;

### Calculate the Proficiency Bonus

In [591]:
ProficiencyBonus=1+np.ceil(Level/4)

In [592]:
ProficiencyBonus=ProficiencyBonus.astype(int)

### Calculate Hit for Spells Attacks casted with Charisma

In [593]:
OtherBonus=1; #Rod of pact keeper

In [594]:
SpellHit=Modifiers[Abilities.index('CHA')]+ProficiencyBonus+OtherBonus

In [595]:
SpellHit

8

### Calculate Hit for Weapon Attacks with Strength

In [596]:
WeaponBonus=1; #Magical +1 weapon

In [597]:
MeleeHit=Modifiers[Abilities.index('STR')]+ProficiencyBonus+WeaponBonus

In [598]:
MeleeHit

8

In [599]:
### Calculate Hit for Weapon Attacks with Dexterity

In [600]:
DexMeleeHit=Modifiers[Abilities.index('DEX')]+ProficiencyBonus+WeaponBonus

In [601]:
DexMeleeHit

6

### The Opponent's Armor Class is:

In [602]:
OAC=15

## Calculate the damage of Eldrich Blast

### Probability to hit an Opponent

#### Probability to hit the Opponent (for 1 dice roll)

In [603]:
PrHit= 1 -(OAC- SpellHit -1)/20 

In [604]:
PrHit

0.7

#### using a function to calculate the probability 
####  probability_of__hitting_opponent(opponent_AC,modifiers,advantage,luck_point,elven_accuracy)

In [605]:
 dndopt.probability_of__hitting_opponent(OAC,SpellHit,0,0,0)

0.7

#### Probability to miss the Opponent (for 1 dice roll)

In [606]:
PrMiss=(OAC- SpellHit -1)/20 

In [607]:
PrMiss

0.3

#### Probability to hit the opponent with advantage (Ad=2)

In [608]:
Ad=2

In [609]:
PrHitA=1-(PrMiss)**(Ad)

In [610]:
PrHitA

0.91

#### testing function

In [611]:
 dndopt.probability_of__hitting_opponent(OAC,SpellHit,1,0,0)

0.91

#### Number of Elrdich Blast Rays

In [612]:
Rays=min(math.floor(Level/5)+1,4)

In [613]:
Rays

2

#### In that case the average damage (per ray) will be (with Agonizing Blast) 1d10+ Charisma Modifier

In [614]:
AverageAgonisingBlastDamage=5.5

In [615]:
AverageDamagePerRay=AverageAgonisingBlastDamage+Modifiers[Abilities.index('CHA')]

In [616]:
AverageDamagePerRay

9.5

Additional modifier for Hex spell

In [617]:
HexAverageDamage=3.5

### Average damage, taking into account critical hits, parametrised by the number of rays

For two rays only the formula would be 

 2 x Probability that only one ray will hit x damage + Probability that both rays will hit x 2 x damage + 2 x Probability that only one dice will crit hit x damage  + Probability that both dices will hit x 2 x damage 

In [618]:
Critit=1/20;

In [619]:
2 * PrHit * PrMiss * AverageDamagePerRay+ 2* (Critit) *(1-Critit) * AverageDamagePerRay+ PrHit**2 * 2* AverageDamagePerRay + 2* (Critit) *(Critit) * AverageDamagePerRay

14.249999999999998

In order to easily generalise to higher number of rays we use the binomial distribution. we check for 2 Rays to see the same result as above.

Without taking into account Crit Hit

In [620]:
ADNCR = scipy.stats.binom.pmf(1,2,PrHit)*AverageDamagePerRay + scipy.stats.binom.pmf(2,2,PrHit)* 2* AverageDamagePerRay 

In [621]:
ADNCR

13.3

Calculating Crit Hit

In [622]:
ACHD= scipy.stats.binom.pmf(1,2,Critit)*AverageDamagePerRay + scipy.stats.binom.pmf(2,2,Critit)* 2* AverageDamagePerRay 

In [623]:
ACHD

0.9500000000000003

Adding the two together

In [624]:
TAD=ADNCR+ACHD

In [625]:
TAD

14.250000000000002

#### Calculating for arbitrary number of Rays

In [626]:
TotalDamage=0;

In [627]:
indexRay = 1
while indexRay < (Rays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,Rays,PrHit)*indexRay*AverageDamagePerRay+scipy.stats.binom.pmf(indexRay,Rays,Critit)*indexRay*AverageDamagePerRay
    indexRay += 1

In [628]:
TotalDamage

14.25

### Average damage with  Advantage

In [629]:
n=1

In [630]:
TotalRays=n*Rays

In [631]:
CrititA=1-(1-1/20)**2

In [632]:
TotalDamage=0

In [633]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHitA)*indexRay*AverageDamagePerRay+scipy.stats.binom.pmf(indexRay,TotalRays,CrititA)*indexRay*AverageDamagePerRay
    indexRay += 1

In [634]:
TotalDamage

19.142500000000005

### Average damage with Advantage and Hex

In [635]:
n=1

In [636]:
TotalRays=n*Rays

In [637]:
TotalDamage=0

In [638]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHitA)*indexRay*(AverageDamagePerRay+HexAverageDamage)+scipy.stats.binom.pmf(indexRay,TotalRays,CrititA)*indexRay*(AverageDamagePerRay+HexAverageDamage)
    indexRay += 1

In [639]:
TotalDamage

26.195000000000004

### Average damage with  n repetitions (e.g. Action Surge n=2)

In [640]:
n=2;

In [641]:
TotalRays=n*Rays

In [642]:
TotalDamage=0

In [643]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHit)*indexRay*AverageDamagePerRay+scipy.stats.binom.pmf(indexRay,TotalRays,Critit)*indexRay*AverageDamagePerRay
    indexRay += 1

In [644]:
TotalDamage

28.500000000000007

### Average damage with  n repetitions  (e.g. Action Surge n=2) and Advantage

For rolls with advantage I need to use in the formula the probability of hitting the target with advantage

In [645]:
n=2;

In [646]:
TotalRays=n*Rays

In [647]:
TotalDamage=0

In [648]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHitA)*indexRay*AverageDamagePerRay+scipy.stats.binom.pmf(indexRay,TotalRays,CrititA)*indexRay*AverageDamagePerRay
    indexRay += 1

In [649]:
  TotalDamage

38.285000000000004

### Average damage with  n repetitions  (e.g. Action Surge n=2), Advantage and Hex

In [650]:
n=2;

In [651]:
TotalRays=n*Rays

In [652]:
TotalRays

4

In [653]:
TotalDamage=0

In [654]:
indexRay = 1
while indexRay < (TotalRays+1):
    TotalDamage+=scipy.stats.binom.pmf(indexRay,TotalRays,PrHitA)*indexRay*(AverageDamagePerRay+HexAverageDamage)+scipy.stats.binom.pmf(indexRay,TotalRays,CrititA)*indexRay*(AverageDamagePerRay+HexAverageDamage)
    indexRay += 1

In [655]:
TotalDamage

52.39

#### Testing the function average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

In [656]:
repetitions=4;average_damage_per_hit=AverageDamagePerRay+HexAverageDamage;opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [657]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

52.39

### Without Action surge, only advantage with two eldrich blast rays (repetitions=2)

In [658]:
repetitions=Rays;average_damage_per_hit=AverageDamagePerRay+HexAverageDamage;opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [659]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

26.195

### With Action Surge and spirit shroud so that I can use the celestial warlock's bonus to radient damage (oddbonus=4);

In [660]:
spirit_shroud=4.5

In [661]:
radient_damage_bonus=Modifiers[Abilities.index('CHA')]# since it is applied only to the first beam, multiply with factor 1/2 to calculate average damage

In [662]:
repetitions=4; average_damage_per_hit=AverageDamagePerRay+HexAverageDamage+spirit_shroud+radient_damage_bonus/2;opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [663]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

78.585

#### Assuming non-celestial warlock (no radient bonus)

In [664]:
repetitions=4; average_damage_per_hit=AverageDamagePerRay+HexAverageDamage+spirit_shroud;opponent_AC=OAC;modifiers=SpellHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [665]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

70.525

## Paladin built: calculate the damage of Melee (+1 weapon) + Hex+ Green Flame Blade+ Radient damage bonus + two Smites with Advantage

#### Magical +1 Long sword damage 1d8+1 (+4) + Green flame blade (calculating for 2 targets) 

In [666]:
WeaponAverageDamage=4.5+1+Modifiers[Abilities.index('STR')]+2*4.5+4

In [667]:
WeaponAverageDamage

22.5

In [668]:
PrHit= 1 -(OAC- MeleeHit -1)/20 

In [669]:
PrHit

0.7

In [670]:
PrMiss=1-PrHit

In [671]:
Ad=2

In [672]:
PrHitAd=1-(PrMiss)**(Ad)

In [673]:
PrHitAd

0.9099999999999999

#### Divine smite (3rd level slot) 4d8 average 4*4.5

In [674]:
Smite=4.5*4

#### bonus smite: I am assuming  searing smite and a celestial warlock's bonus to radient damage

In [675]:
BonusSmite=3*3.5+Modifiers[Abilities.index('CHA')]

In [676]:
HexAverageDamage=3.5 # will cast it with a 1st level paladine slot

In [677]:
TotalDamage= (WeaponAverageDamage+Smite+BonusSmite+HexAverageDamage)*PrHitAd+ (WeaponAverageDamage+Smite+BonusSmite+HexAverageDamage) *CrititA

In [678]:
TotalDamage

58.93874999999999

#### Using the function average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

In [679]:
repetitions=1;average_damage_per_hit=WeaponAverageDamage+Smite+BonusSmite+HexAverageDamage;opponent_AC=OAC;modifiers=MeleeHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [680]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

58.93875

#### assuming non-celestial warlock

In [681]:
BonusSmite=3*3.5

In [682]:
repetitions=1;average_damage_per_hit=WeaponAverageDamage+Smite+BonusSmite+HexAverageDamage;opponent_AC=OAC;modifiers=MeleeHit;advantage=1;luck_point=0;elven_accuracy=0;hexblade_curse=0

In [683]:
dndopt.average_damage(repetitions,average_damage_per_hit,opponent_AC,modifiers,advantage,luck_point,elven_accuracy,hexblade_curse)

54.90875

## Higher than the fighter (when Eldrich Blast only is considered) but asks for the consumption of two spell slots of the highest level (+1 first level). The characters casts first searing smite as bonus action. Slightly more complecated mechanics. If the Fighter combo consumes one slot with spirit shroud (radient) and one with Hex then I can exploit the Celestial Warlock Bonus and surpass the Paladin combo. At this level, I would not have another slot as a Paladin to also cast spirit shroud. The fighter combo suprasses the paladin combo at this level also without the Celestial Warlock Bonus.